# Part A

In [2]:
# Libraries Import
import pandas as pd
import numpy as np
import statistics
import keras
from keras.models import Sequential
from keras.layers import Dense

Using TensorFlow backend.
/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:520: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:521: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/jupyterlab/con

In [3]:
# Data Import and consistency check
concrete_data = pd.read_csv('https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0101EN/labs/data/concrete_data.csv')
concrete_data.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28,79.99
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28,61.89
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270,40.27
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365,41.05
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360,44.30


In [4]:
concrete_data.shape

(1030, 9)

In [5]:
concrete_data.describe()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
count,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000
mean,281.167864,73.895825,54.188350,181.567282,6.204660,972.918932,773.580485,45.662136,35.817961
std,104.506364,86.279342,63.997004,21.354219,5.973841,77.753954,80.175980,63.169912,16.705742
min,102.000000,0.000000,0.000000,121.800000,0.000000,801.000000,594.000000,1.000000,2.330000
25%,192.375000,0.000000,0.000000,164.900000,0.000000,932.000000,730.950000,7.000000,23.710000
50%,272.900000,22.000000,0.000000,185.000000,6.400000,968.000000,779.500000,28.000000,34.445000
75%,350.000000,142.950000,118.300000,192.000000,10.200000,1029.400000,824.000000,56.000000,46.135000
max,540.000000,359.400000,200.100000,247.000000,32.200000,1145.000000,992.600000,365.000000,82.600000


In [6]:
concrete_data.isnull().sum()

Cement                0
Blast Furnace Slag    0
Fly Ash               0
Water                 0
Superplasticizer      0
Coarse Aggregate      0
Fine Aggregate        0
Age                   0
Strength              0
dtype: int64

#### The data is clean, and no null value is present.

In [7]:
# Split data into predictors and target
concrete_data_columns = concrete_data.columns

predictors = concrete_data[concrete_data_columns[concrete_data_columns != 'Strength']] # all columns except Strength
target = concrete_data['Strength'] # Strength column

In [8]:
predictors.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360


In [9]:
target.head()

0    79.99
1    61.89
2    40.27
3    41.05
4    44.30
Name: Strength, dtype: float64

In [10]:
# Number of predictors definition
n_cols = predictors.shape[1] # number of predictors
n_cols

8

In [11]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(predictors, target, test_size=0.30, random_state=1)

print("number of test samples :", X_test.shape[0])
print("number of training samples:",X_train.shape[0])

number of test samples : 309
number of training samples: 721


In [12]:
# Definition of the regression model
def regression_model():
    # create model
    model = Sequential()
    model.add(Dense(10, activation='relu', input_shape=(n_cols,)))
    model.add(Dense(1))
    
    # compile model
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

In [13]:
# List for mean squared error cumulation
scores = []

# looping 50 times over model
for i in range(50):

    # build the model
    model = regression_model()

    # fit the model
    model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=50, verbose=2)

    # evaluate the model
    score = model.evaluate(X_test, y_test, verbose=0)
    scores.append(score)

Train on 721 samples, validate on 309 samples
Epoch 1/50
 - 1s - loss: 24779.6933 - val_loss: 10381.0831
Epoch 2/50
 - 0s - loss: 6536.3458 - val_loss: 2735.1318
Epoch 3/50
 - 0s - loss: 2289.2400 - val_loss: 1675.6856
Epoch 4/50
 - 0s - loss: 1692.2402 - val_loss: 1603.2346
Epoch 5/50
 - 0s - loss: 1596.9769 - val_loss: 1526.8375
Epoch 6/50
 - 0s - loss: 1524.2548 - val_loss: 1455.1049
Epoch 7/50
 - 0s - loss: 1452.1325 - val_loss: 1370.5518
Epoch 8/50
 - 0s - loss: 1384.3441 - val_loss: 1298.9218
Epoch 9/50
 - 0s - loss: 1316.1288 - val_loss: 1240.0100
Epoch 10/50
 - 0s - loss: 1257.5523 - val_loss: 1181.5415
Epoch 11/50
 - 0s - loss: 1202.9521 - val_loss: 1124.3848
Epoch 12/50
 - 0s - loss: 1153.5132 - val_loss: 1074.9095
Epoch 13/50
 - 0s - loss: 1099.9009 - val_loss: 1022.8925
Epoch 14/50
 - 0s - loss: 1055.2321 - val_loss: 977.2375
Epoch 15/50
 - 0s - loss: 1009.9259 - val_loss: 931.1719
Epoch 16/50
 - 0s - loss: 967.5926 - val_loss: 895.1758
Epoch 17/50
 - 0s - loss: 928.9972 - 

In [14]:
# Score list of non-normalized data
scores

[257.27468664669294,
 158.0313089611461,
 114.44530166082784,
 204.7983038448593,
 130.87383697411002,
 128.52655310769683,
 1758.7294479419497,
 128.57808174207372,
 143.5115615696583,
 432.99278738274927,
 220.89780280350868,
 1883.7605225399475,
 126.63074999565445,
 227.87925083197436,
 203.3509749131681,
 1606.0946949585357,
 143.0665423445717,
 283.733326538481,
 365.8406340466348,
 3104.0019855190635,
 78.33314886988174,
 500.110491805092,
 338.5622602049201,
 120.58742046664834,
 172.44508198858466,
 158.0992879034246,
 178.88329105315472,
 363.74196321061515,
 3120.7045937942457,
 406.2003375303398,
 408.8188109166414,
 314.02915692869516,
 125.3498228252127,
 209.1077792960849,
 213.177110937421,
 677.94074928259,
 384.8008683584269,
 193.59538293733567,
 139.75648236814826,
 918.4589136611296,
 176.22765520706918,
 168.63667317115758,
 156.0118837819516,
 147.51745015363477,
 155.74308618835647,
 115.33009488991549,
 123.04299388502794,
 133.43468944463143,
 697.322967331772

In [15]:
print("The mean is", round(statistics.mean(scores),1), "and the standard deviation is" , round(statistics.stdev(scores),1))

The mean is 454.7 and the standard deviation is 676.3


# Part B

### Normalizing the data.

In [16]:
predictors_norm = (predictors - predictors.mean()) / predictors.std()
predictors_norm.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age
0,2.476712,-0.856472,-0.846733,-0.916319,-0.620147,0.862735,-1.217079,-0.279597
1,2.476712,-0.856472,-0.846733,-0.916319,-0.620147,1.055651,-1.217079,-0.279597
2,0.491187,0.795140,-0.846733,2.174405,-1.038638,-0.526262,-2.239829,3.551340
3,0.491187,0.795140,-0.846733,2.174405,-1.038638,-0.526262,-2.239829,5.055221
4,-0.790075,0.678079,-0.846733,0.488555,-1.038638,0.070492,0.647569,4.976069


In [17]:
from sklearn.model_selection import train_test_split

X_train_b, X_test_b, y_train_b, y_test_b = train_test_split(predictors_norm, target, test_size=0.30, random_state=1)

print("number of test samples :", X_test_b.shape[0])
print("number of training samples:",X_train_b.shape[0])

number of test samples : 309
number of training samples: 721


In [18]:
# List for mean squared error cumulation of normalized data
scores_b = []

# looping 50 times over model
for i in range(50):

    # build the model
    model = regression_model()

    # fit the model
    model.fit(X_train_b, y_train_b, validation_data=(X_test_b, y_test_b), epochs=50, verbose=2)

    # evaluate the model
    score_b = model.evaluate(X_test_b, y_test_b, verbose=0)
    scores_b.append(score_b)

Train on 721 samples, validate on 309 samples
Epoch 1/50
 - 4s - loss: 1492.4246 - val_loss: 1507.0378
Epoch 2/50
 - 0s - loss: 1474.2634 - val_loss: 1487.9177
Epoch 3/50
 - 0s - loss: 1456.0949 - val_loss: 1468.0155
Epoch 4/50
 - 0s - loss: 1437.4580 - val_loss: 1448.0410
Epoch 5/50
 - 0s - loss: 1418.2941 - val_loss: 1427.7134
Epoch 6/50
 - 0s - loss: 1398.8006 - val_loss: 1406.3649
Epoch 7/50
 - 0s - loss: 1378.4087 - val_loss: 1383.6490
Epoch 8/50
 - 0s - loss: 1356.5449 - val_loss: 1360.6855
Epoch 9/50
 - 0s - loss: 1334.2189 - val_loss: 1335.9520
Epoch 10/50
 - 0s - loss: 1310.4483 - val_loss: 1310.4407
Epoch 11/50
 - 0s - loss: 1285.5252 - val_loss: 1283.4249
Epoch 12/50
 - 0s - loss: 1259.5193 - val_loss: 1255.2288
Epoch 13/50
 - 0s - loss: 1232.6494 - val_loss: 1225.8912
Epoch 14/50
 - 0s - loss: 1204.4558 - val_loss: 1196.1154
Epoch 15/50
 - 0s - loss: 1175.7357 - val_loss: 1165.5184
Epoch 16/50
 - 0s - loss: 1146.1141 - val_loss: 1134.0806
Epoch 17/50
 - 0s - loss: 1115.8653

In [19]:
print("The mean of normalized data is", round(statistics.mean(scores_b),1), "and the standard deviation is" , round(statistics.stdev(scores_b),1))

The mean of normalized data is 386.5 and the standard deviation is 150.1


# Part C

In [20]:
# List for mean squared error cumulation of normalized data for 100 epochs
scores_c = []

# looping 50 times over model
for i in range(50):

    # build the model
    model = regression_model()

    # fit the model
    model.fit(X_train_b, y_train_b, validation_data=(X_test_b, y_test_b), epochs=100, verbose=2)

    # evaluate the model
    score_c = model.evaluate(X_test_b, y_test_b, verbose=0)
    scores_c.append(score_c)

Train on 721 samples, validate on 309 samples
Epoch 1/100
 - 8s - loss: 1478.6383 - val_loss: 1499.0280
Epoch 2/100
 - 0s - loss: 1462.8066 - val_loss: 1482.3487
Epoch 3/100
 - 0s - loss: 1446.4260 - val_loss: 1464.4985
Epoch 4/100
 - 0s - loss: 1428.7912 - val_loss: 1445.7434
Epoch 5/100
 - 0s - loss: 1410.3587 - val_loss: 1425.5016
Epoch 6/100
 - 0s - loss: 1390.4961 - val_loss: 1404.0571
Epoch 7/100
 - 0s - loss: 1369.2671 - val_loss: 1381.5168
Epoch 8/100
 - 0s - loss: 1347.0311 - val_loss: 1357.5911
Epoch 9/100
 - 0s - loss: 1323.3065 - val_loss: 1331.9770
Epoch 10/100
 - 0s - loss: 1298.0373 - val_loss: 1304.8924
Epoch 11/100
 - 0s - loss: 1271.2925 - val_loss: 1276.5154
Epoch 12/100
 - 0s - loss: 1242.6659 - val_loss: 1247.0018
Epoch 13/100
 - 0s - loss: 1213.0808 - val_loss: 1214.8214
Epoch 14/100
 - 0s - loss: 1181.2988 - val_loss: 1181.4098
Epoch 15/100
 - 0s - loss: 1148.3992 - val_loss: 1147.1742
Epoch 16/100
 - 0s - loss: 1114.0779 - val_loss: 1111.9538
Epoch 17/100
 - 0s 

In [22]:
print("The mean of normalized data for 100 epochs is", round(statistics.mean(scores_c),1), "and the standard deviation is" , round(statistics.stdev(scores_c),1))

The mean of normalized data for 100 epochs is 187.1 and the standard deviation is 15.1


# Part D

In [23]:
# Definition of the regression model with three hidden layer
def regression_model_d():
    # create model
    model = Sequential()
    model.add(Dense(10, activation='relu', input_shape=(n_cols,)))
    model.add(Dense(10, activation='relu'))
    model.add(Dense(10, activation='relu'))
    model.add(Dense(1))
    
    # compile model
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

In [24]:
# List for mean squared error cumulation of normalized data (three hidden layers model)
scores_d = []

# looping 50 times over model
for i in range(50):

    # build the model
    model = regression_model_d()

    # fit the model
    model.fit(X_train_b, y_train_b, validation_data=(X_test_b, y_test_b), epochs=50, verbose=2)

    # evaluate the model
    score_d = model.evaluate(X_test_b, y_test_b, verbose=0)
    scores_d.append(score_d)

Train on 721 samples, validate on 309 samples
Epoch 1/50
 - 12s - loss: 1551.4510 - val_loss: 1583.7812
Epoch 2/50
 - 0s - loss: 1542.3773 - val_loss: 1573.6355
Epoch 3/50
 - 0s - loss: 1531.0295 - val_loss: 1559.0687
Epoch 4/50
 - 0s - loss: 1513.7248 - val_loss: 1535.5382
Epoch 5/50
 - 1s - loss: 1485.8259 - val_loss: 1496.0582
Epoch 6/50
 - 0s - loss: 1438.9259 - val_loss: 1432.3977
Epoch 7/50
 - 0s - loss: 1365.9374 - val_loss: 1337.1577
Epoch 8/50
 - 1s - loss: 1256.5039 - val_loss: 1192.5789
Epoch 9/50
 - 0s - loss: 1090.4551 - val_loss: 980.0163
Epoch 10/50
 - 1s - loss: 863.7362 - val_loss: 715.3809
Epoch 11/50
 - 0s - loss: 599.1124 - val_loss: 472.0268
Epoch 12/50
 - 0s - loss: 379.8929 - val_loss: 329.1228
Epoch 13/50
 - 0s - loss: 265.3643 - val_loss: 291.8363
Epoch 14/50
 - 0s - loss: 232.6213 - val_loss: 279.0222
Epoch 15/50
 - 0s - loss: 216.9006 - val_loss: 260.5154
Epoch 16/50
 - 0s - loss: 205.8318 - val_loss: 251.1298
Epoch 17/50
 - 0s - loss: 197.4215 - val_loss: 24